<a href="https://colab.research.google.com/github/Aatika1/openai-API/blob/main/Hybrid_Search_RAG_With_Langchain_And_Pinecone_Vector_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade --quiet pinecone-client pinecone-text pinecone-notebooks

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 7.0 MB/s eta 0:00:00


In [2]:
!pip install --upgrade --quiet langchain pinecone-client google-generativeai openai tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 24.8 MB/s eta 0:00:00


In [3]:
!pip install --upgrade --quiet langchain_community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.2 MB/s eta 0:00:00


In [4]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [5]:
from google.colab import userdata
pinecone_api_key=userdata.get('pinecone_api_key')

In [6]:
import os
from pinecone import Pinecone,ServerlessSpec
index_name = "hybird-search-langchain-pinecone"
# initialize the pinecone client
pc=Pinecone(api_key=pinecone_api_key)

#create the index
if index_name not in pc.list_indexes().names():
  pc.create_index(
      name = index_name,
      dimension=384, #dimension of dense vector
      metric='dotproduct', # sparse values supporlisted only for dotproduct
      spec=ServerlessSpec(cloud='aws',region='us-east-1'),

  )

In [8]:
index=pc.Index(index_name)
index

In [14]:
!pip install langchain transformers


In [22]:
from dotenv import load_dotenv

load_dotenv()
# os.environ['HF_TOKEN']=userdata.get('HF_TOKEN')

from langchain.embeddings import HuggingFaceEmbeddings

embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [31]:
from pinecone_text.sparse import BM25Encoder
bm25_encoder = BM25Encoder().default()
bm25_encoder


In [33]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [35]:
sentences=[
    "In 2023 I visted in karbala",
    "In 2022 I visted in syria",
    "In 2021 I visted in makka and madina",
]

#tfidf values on these sentences

bm25_encoder.fit(sentences)

#store the values to json file
bm25_encoder.dump("bm25_values.json")

##load to your BM25encoder object
bm25_encoder=BM25Encoder().load("bm25_values.json")


  0%|          | 0/3 [00:00<?, ?it/s]

In [36]:
retriever=PineconeHybridSearchRetriever(embeddings=embedding,sparse_encoder=bm25_encoder,index=index)


In [37]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x7feb1a420eb0>, index=<pinecone.data.index.Index object at 0x7feb1a66f7c0>)

In [38]:
retriever.add_texts(texts=["In 2023 I visted in karbala",
                           "In 2022 I visted in syria",
                           "In 2021 I visted in makka and madina"])

  0%|          | 0/1 [00:00<?, ?it/s]

In [43]:
question=retriever.invoke("which city I visited 1st")
print(question)

[Document(metadata={'score': 0.0914095789}, page_content='In 2021 I visted in makka and madina'), Document(metadata={'score': 0.0911798626}, page_content='In 2023 I visted in karbala'), Document(metadata={'score': 0.0631975532}, page_content='In 2022 I visted in syria')]
